özellik çıkarımı
ham veriyi doğrudan kullanmak yerine bir veri kümesinden modelin öğrenmesini kolaylaştıracak anlamlı ve özet bilgiler çıkarmak anlamına gelir.
2 ye ayrılır
1. yapısal veilerden değişken türetmek = zaten belli yapıdaki değişkenlerden yeni değişken üretmek
2. yapısal olmayan verilerden değişken türetmek = ses görüntü gibi metin gibi yapısal olmayan verilerden numerik değişken türetmek

mesela yapısal olmayan değişken örneği:
7:46 2/14/25  --> ten saat gün ay yıl değişkenleri çıkarılabilir veya hafatanın hangi gününe denk geldiği bilgiside çıkar
veya bir görseli makinanın anlayabileceği formata çevirmek için deep learningde birsürü farklı matematiğe dökme yöntemi vardır


### önce olası tüm feature ları çıkart sonra değerlendirme yap bu olur bu olmaz diye çünkü ne çıkacağı belli olmaz

In [100]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: "%.3f" %x)
pd.set_option("display.Width", 500)


In [101]:
# binary features

df = pd.read_csv("titanic.csv")
# burada varolan değişkenleri değiştirmiyoruz
# varolan değişkenden yeni bir değişken türetiyoruz

# mesela titanicteki cabin değişkeninde bazı eksik değerlere 1 bazılarına 0 yazacağım (normalde cabin çöp değişken çok fazla eksik var)
df["NEW_CABIN_BOOL"] = df["Cabin"].notnull().astype("int") # dolu ise int yap ve yeni değişkene ata
df.head() # eksik değere 0 dedi dolu değerlere ise 1 dedi artık anlamsız olan cabin değişkeni daha anlamlı hale geldi

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_CABIN_BOOL
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,0


In [102]:
df.groupby("NEW_CABIN_BOOL").agg({"Survived": "mean"}) # yeni değişkenimizi survived a göre groupbya alıp
# hayatta kalanların yaşama ortalamasına bakıyorum (ki cabini null olanlar genelde gemi personeliydi)
# demekki personelleri harcamışlar çakkallar

,Survived
NEW_CABIN_BOOL,
0,0.300
1,0.667


In [103]:
# şimdide bu iki oranı bi kıyaslayalım
from statsmodels.stats.proportion import proportions_ztest

count = [
    df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].sum(),  # NEW_CABIN_BOOL = 1 olanların kaç tanesi hayatta
    df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].sum()   # NEW_CABIN_BOOL = 0 olanların kaç tanesi hayatta
]

nobs = [
    df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].shape[0],  # NEW_CABIN_BOOL = 1 olanların toplam sayısı
    df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].shape[0]   # NEW_CABIN_BOOL = 0 olanların toplam sayısı
]

test_stat, pvalue = proportions_ztest(count, nobs)
print(f"Z-Test İstatistiği: {test_stat:.4f}, p-değeri: {pvalue:.4f}")


Z-Test İstatistiği: 9.4597, p-değeri: 0.0000


çıktıdaki z-test statistiği şudur = new_cabin_bool 1 oarnı new_cabin_bool 0 oranından yüksek 9.45 Genellikle |Z| > 2 ise fark anlamlı kabul edilir. yani kabini olanlar daha çok hayatta kalmışlar

p-değeri ise Bu fark tamamen şans eseri olabilir mi?" sorusuna yanıt verir.
Genellikle α (alfa) = 0.05 eşiği kullanılır (yani %5 hata payı kabul edilir).
p-değeri < 0.05 ise: İki grup arasındaki fark istatistiksel olarak anlamlıdır, yani bu fark tesadüfi değildir.
sözün özü p değeri yüzde 5 altında ise bu new cabin bool değişkenin anlamlı birşeye dönüşmüş burdan yürü

In [104]:
# sibSp kardeşi eşi varmı değişkeni parch ise gemide ebeveyni veya çocuğu varmı sorusu
df.loc[((df["SibSp"] + df["Parch"]) > 0), "NEW_IS_ALONE"] = "NO" # yani gemide herhangi bir akraba varsa yalnız değil
df.loc[((df["SibSp"] + df["Parch"]) == 0), "NEW_IS_ALONE"] = "YES" # yoksada yalnız
# bi bakalım belki kişinin akrabası olması durumu hayatta kalma savaşını güçlendirmiştir bi görelim

df.groupby("NEW_IS_ALONE").agg({"Survived": "mean"}) # şimdilik hayatta kalmayla eşi dostu olmanın bir bağlantısı var gibi gözüküyor
# ama birde istatistiki olarak bakalım

,Survived
NEW_IS_ALONE,
NO,0.506
YES,0.304


In [105]:
from statsmodels.stats.proportion import proportions_ztest

count = [
    df.loc[df["NEW_IS_ALONE"] == "YES", "Survived"].sum(),  
    df.loc[df["NEW_IS_ALONE"] == "NO", "Survived"].sum()   
]

nobs = [
    df.loc[df["NEW_IS_ALONE"] == "YES", "Survived"].shape[0],  
    df.loc[df["NEW_IS_ALONE"] == "NO", "Survived"].shape[0] 
]

test_stat, pvalue = proportions_ztest(count, nobs)
print(f"Z-Test İstatistiği: {test_stat:.4f}, p-değeri: {pvalue:.4f}")

# p değerine göre evet çıkarım doğru anlam varmış

Z-Test İstatistiği: -6.0704, p-değeri: 0.0000


# ------------------------------------------------------------------------------------------------
### textler üzerinden özellik türetmek
# ------------------------------------------------------------------------------------------------

In [106]:
dff = pd.read_csv("titanic.csv") # yeni baştan çağırıyorumki en temiz hali gelsin
dff.head() # amacımız burada name kategorisinden anlamlı değişkenler çıkarmı sorusuna cevap vermek

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


In [107]:
# mesela harfleri çıkartmaya çalışalım ---LETTER COUNT---
df["NEW_NAME_COUNT"] = df["Name"].str.len()

# birde kelimeleri bir sayalım ---WORD COUNT---
df["NEW_NAME_COUNT"] = df["Name"].apply(lambda x: len(str(x).split(" ")))

# birde özel yapıları yakalamaya çalışalım 
# mesela doktor ları bir görelim
df["NEW_NAME_DR"] = df["Name"].apply(lambda x: len([x for x in x.split() if x.startswith("Dr")]))
df.groupby("NEW_NAME_DR").agg({"Survived": "mean"})
# doktorların hayatta kalma olasılığı daha yüksek gözüküyor

,Survived
NEW_NAME_DR,
0,0.383
1,0.500


# ------------------------------------------------------------------------------------------------
### regex ile değişken türetmek
# ------------------------------------------------------------------------------------------------

In [108]:
# mesela burada verisetindeki isim sütunundaki bütün title ları bir çekelim bakalım kimlerden kaç kişi varmış
dff.head()
# mesela Miss. Dr. Lady. gibi title ları ayıralım

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


In [109]:
df["NEW_TİTLE"] = df.Name.str.extract(" ([A-Za-z]+)\.", expand=False) # extract belirttiğim şeyi çıkarmamı sağlar

# şimdide groupby a alalım ve inceleyelim
df[["NEW_TİTLE", "Survived", "Age"]].groupby(["NEW_TİTLE"]).agg({"Survived": "mean", "Age": ["count", "mean"]})

# herkes geldi kaptan reis gemiyle birlikte batmış (r.i.p)
# kocalar hanımlarına değer vermiş kral adamlarmış
# çakkal soylu bitane varmış oda kendini kurtarmış

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_83785/2995009554.py:1: SyntaxWarning: invalid escape sequence '\.'
  df["NEW_TİTLE"] = df.Name.str.extract(" ([A-Za-z]+)\.", expand=False) # extract belirttiğim şeyi çıkarmamı sağlar


Survived   Age       
              mean count   mean
NEW_TİTLE                      
Capt         0.000     1 70.000
Col          0.500     2 58.000
Countess     1.000     1 33.000
Don          0.000     1 40.000
Dr           0.429     6 42.000
Jonkheer     0.000     1 38.000
Lady         1.000     1 48.000
Major        0.500     2 48.500
Master       0.575    36  4.574
Miss         0.698   146 21.774
Mlle         1.000     2 24.000
Mme          1.000     1 24.000
Mr           0.157   398 32.368
Mrs          0.792   108 35.898
Ms           1.000     1 28.000
Rev          0.000     6 43.167
Sir          1.000     1 49.000

In [110]:
# date değişkenleri üretelim
df = pd.read_csv("course_reviews.csv")
df.head()

# şimdi bu timestamp değişkeninden yeni değişkenler üretelim

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered
0,5.000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.000,0.000,0.000
1,5.000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.000,0.000,0.000
2,4.500,2021-02-04 20:34:03,2019-07-04 23:23:27,1.000,0.000,0.000
3,5.000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.000,0.000,0.000
4,4.000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.000,0.000,0.000


In [111]:
df.info() # timestamp object tipinde bunu değiştirelim

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rating              4323 non-null   float64
 1   Timestamp           4323 non-null   object 
 2   Enrolled            4323 non-null   object 
 3   Progress            4323 non-null   float64
 4   Questions Asked     4323 non-null   float64
 5   Questions Answered  4323 non-null   float64
dtypes: float64(4), object(2)
memory usage: 202.8+ KB


In [112]:
df["Timestamp"] = pd.to_datetime(df["Timestamp"], format="%Y-%m-%d %H:%M:%S")
# to datetime bana soruyor yıl gün ay hangi sırada diye onu belirttim
df.info() # artık object değil

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Rating              4323 non-null   float64       
 1   Timestamp           4323 non-null   datetime64[ns]
 2   Enrolled            4323 non-null   object        
 3   Progress            4323 non-null   float64       
 4   Questions Asked     4323 non-null   float64       
 5   Questions Answered  4323 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 202.8+ KB


In [113]:
df["month"] = df["Timestamp"].dt.month # mesela ay ı atadık
df["year"] = df["Timestamp"].dt.year # mesela yıl ı atadık
df["year_diff"] = date.today().year - df["Timestamp"].dt.year # bulunduğumuz yıl ile verisetindeki yılları çıkart kaç yıl geçtiğini bul


# ------------------------------------------------------------------------------------------------
### feature interactions (özellik etkileşimleri)
# ------------------------------------------------------------------------------------------------

mesela iki değişkenin toplamı farkı karesi kökü vb...
ama bunların anlamlı şeyler olması lazım
örnek aşağıda

In [114]:
df = pd.read_csv("titanic.csv")
df["NEW_AGE_PCLASS"] = df["Age"] * df["Pclass"] # mesela burada 
# yaşı küçük olan first class yolculuk yapıyor olabilir
# veya yaşı büyük olan 3. sınıf yolculuk yapıyor olabilir 
# bunlar arasındaki ilişki nedir diye sorulursa refah farkı var birisi genç yaşta zengin sınıfı
# diğeri olgun ama fakir sınıfı bunlar nasıl hayatta kalmış diye bakılabilir


df["NEW_FAMİLY_SİZE"] = df["SibSp"] + df["Parch"] + 1 # akraba sayısı + kişinin kendisi (+1) dersek
# torpilli aile varmı diye bakılabilir

In [115]:
df.loc[(df["Sex"] == "female") & (df["Age"] > 50), "NEW_SEX_CAT"] = "senior_female"
df.loc[(df["Sex"] == "female") & ((df["Age"] > 21) & (df["Age"] < 50)), "NEW_SEX_CAT"] = "mature_female"

df.loc[(df["Sex"] == "male") & (df["Age"] > 50), "NEW_SEX_CAT"] = "senior_male"
df.groupby("NEW_SEX_CAT")["Survived"].mean()
# mesela buradan yine görüyoruz anlatmaya gerek yok

NEW_SEX_CAT
mature_female   0.774
senior_female   0.941
senior_male     0.128
Name: Survived, dtype: float64